# Feature Extraction from Time Series Data Using a Neural Network

Currently doing this without pycbc as the docker doesnt have pytorch installed yet

TODO:
- Generate simple data
- Add noise
- Feature extraction using NN

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# just generating some datasets for the time being
data_samples = 1000
mean = 10
std = 2
no_time_steps = 4000

amplitudes = np.random.normal(mean, std, data_samples)

time_steps = np.linspace(0,120,no_time_steps)

signals = np.array([np.sin(time_steps)*a for a in amplitudes])

# adding noise, just gaussian for the time being
noise_mean = 1
noise_std = 2
noised_signals = np.array([x + np.random.normal(noise_mean, noise_std, no_time_steps) for x in signals])


In [49]:
x_train, x_test, y_train, y_test = train_test_split(noised_signals, amplitudes, test_size=0.2)